In [ ]:
# Amazon Sales Data Cleaning

This notebook will clean and prepare the Amazon sales dataset for analysis in Power BI. We'll go through the following steps:

1. Data Loading and Initial Inspection
2. Data Type Conversion
3. Cleaning Individual Columns
4. Adding Calculated Columns
5. Handling Missing Values
6. Data Validation
7. Exporting Clean Data


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Read the dataset
df = pd.read_csv('amazon.csv')

# Display basic information
print("Dataset Shape:", df.shape)
print("\nColumns in the dataset:")
print("-" * 50)
for col in df.columns:
    print(f"{col}: {df[col].dtype}")

print("\nFirst few rows of the dataset:")
display(df.head())

# Check for missing values
print("\nMissing values in each column:")
print("-" * 50)
print(df.isnull().sum())


In [ ]:
## Data Cleaning Steps

### 1. Clean Price Columns
- Convert price columns to numeric
- Remove currency symbols
- Handle any invalid values


In [ ]:
# Display sample of price columns before cleaning
print("Sample of price values before cleaning:")
print("-" * 50)
print("\nDiscounted Price samples:", df['discounted_price'].head())
print("\nActual Price samples:", df['actual_price'].head())

# Clean price columns
def clean_price(price):
    if isinstance(price, str):
        # Remove ₹ symbol and commas, then convert to float
        return float(price.replace('₹', '').replace(',', '').strip())
    return float(price)

# Apply cleaning to price columns
df['discounted_price'] = df['discounted_price'].apply(clean_price)
df['actual_price'] = df['actual_price'].apply(clean_price)

# Display sample after cleaning
print("\nSample of price values after cleaning:")
print("-" * 50)
print("\nDiscounted Price samples:", df['discounted_price'].head())
print("\nActual Price samples:", df['actual_price'].head())

# Basic statistics of price columns
print("\nPrice Statistics:")
print("-" * 50)
print(df[['discounted_price', 'actual_price']].describe())


In [ ]:
### 2. Clean Discount Percentage
- Remove % symbol
- Convert to numeric
- Validate values are between 0 and 100


In [ ]:
# Display sample of discount percentage before cleaning
print("Sample of discount percentage before cleaning:")
print("-" * 50)
print(df['discount_percentage'].head())

# Clean discount percentage
df['discount_percentage'] = df['discount_percentage'].str.rstrip('%').astype(float)

# Validate discount percentage values
print("\nDiscount Percentage Statistics:")
print("-" * 50)
print(df['discount_percentage'].describe())

# Visualize discount distribution
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='discount_percentage', bins=30)
plt.title('Distribution of Discount Percentages')
plt.xlabel('Discount Percentage')
plt.ylabel('Count')
plt.show()

# Check for any invalid values (negative or >100)
invalid_discounts = df[
    (df['discount_percentage'] < 0) | 
    (df['discount_percentage'] > 100)
]
if len(invalid_discounts) > 0:
    print("\nFound invalid discount percentages:")
    print(invalid_discounts[['product_name', 'discount_percentage']])


In [ ]:
### 3. Clean Rating and Rating Count
- Convert ratings to numeric values
- Clean rating count (remove text, commas)
- Handle missing values


In [ ]:
# Display samples before cleaning
print("Samples before cleaning:")
print("-" * 50)
print("\nRating samples:", df['rating'].head())
print("\nRating count samples:", df['rating_count'].head())

# Clean rating
def clean_rating(x):
    try:
        return float(str(x).split()[0])
    except (ValueError, IndexError):
        return None

# Clean rating_count
def clean_rating_count(x):
    try:
        # Remove commas and any text, keep only numbers
        num = ''.join(c for c in str(x) if c.isdigit())
        return int(num) if num else 0
    except (ValueError, TypeError):
        return 0

# Apply cleaning
df['rating'] = df['rating'].apply(clean_rating)
df['rating_count'] = df['rating_count'].apply(clean_rating_count)

# Display samples after cleaning
print("\nSamples after cleaning:")
print("-" * 50)
print("\nRating samples:", df['rating'].head())
print("\nRating count samples:", df['rating_count'].head())

# Statistics after cleaning
print("\nRating Statistics:")
print("-" * 50)
print(df[['rating', 'rating_count']].describe())

# Visualize rating distribution
plt.figure(figsize=(15, 5))

plt.subplot(121)
sns.histplot(data=df, x='rating', bins=10)
plt.title('Distribution of Ratings')
plt.xlabel('Rating')
plt.ylabel('Count')

plt.subplot(122)
sns.histplot(data=df, x='rating_count', bins=30)
plt.title('Distribution of Rating Counts')
plt.xlabel('Number of Ratings')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Check for any invalid ratings
invalid_ratings = df[
    (df['rating'].isnull()) | 
    (df['rating'] < 0) | 
    (df['rating'] > 5)
]
if len(invalid_ratings) > 0:
    print("\nFound invalid ratings:")
    print(invalid_ratings[['product_name', 'rating']])


In [ ]:
### 4. Add Calculated Columns
- Calculate savings amount
- Calculate savings percentage
- Create price range categories
- Create discount range categories


In [ ]:
# Calculate savings
df['savings_amount'] = df['actual_price'] - df['discounted_price']
df['savings_percentage'] = (df['savings_amount'] / df['actual_price'] * 100).round(2)

# Create price range categories
def get_price_range(price):
    if price <= 500:
        return "Budget (≤₹500)"
    elif price <= 2000:
        return "Mid-Range (₹501-₹2000)"
    elif price <= 5000:
        return "Premium (₹2001-₹5000)"
    else:
        return "Luxury (>₹5000)"

df['price_range'] = df['discounted_price'].apply(get_price_range)

# Create discount range categories
def get_discount_range(discount):
    if discount <= 10:
        return "Low (≤10%)"
    elif discount <= 30:
        return "Medium (11-30%)"
    elif discount <= 50:
        return "High (31-50%)"
    else:
        return "Very High (>50%)"

df['discount_range'] = df['discount_percentage'].apply(get_discount_range)

# Display sample of new columns
print("Sample of calculated columns:")
print("-" * 50)
display(df[['product_name', 'savings_amount', 'savings_percentage', 
           'price_range', 'discount_range']].head())

# Distribution of products across price ranges
plt.figure(figsize=(12, 5))

plt.subplot(121)
price_range_counts = df['price_range'].value_counts()
sns.barplot(x=price_range_counts.values, y=price_range_counts.index)
plt.title('Distribution of Products by Price Range')
plt.xlabel('Number of Products')

plt.subplot(122)
discount_range_counts = df['discount_range'].value_counts()
sns.barplot(x=discount_range_counts.values, y=discount_range_counts.index)
plt.title('Distribution of Products by Discount Range')
plt.xlabel('Number of Products')

plt.tight_layout()
plt.show()

# Summary statistics of savings
print("\nSavings Statistics:")
print("-" * 50)
print(df[['savings_amount', 'savings_percentage']].describe())


In [ ]:
### 5. Handle Missing Values and Duplicates
- Check for missing values in all columns
- Remove or fill missing values appropriately
- Identify and handle duplicate entries


In [ ]:
# Check missing values
print("Missing values in each column:")
print("-" * 50)
print(df.isnull().sum())

# Handle missing values
# Fill missing ratings with median
df['rating'].fillna(df['rating'].median(), inplace=True)

# Fill missing rating counts with 0
df['rating_count'].fillna(0, inplace=True)

# Check for duplicate products
duplicates = df[df.duplicated(subset=['product_name', 'discounted_price'], keep=False)]
print("\nNumber of duplicate products:", len(duplicates))

if len(duplicates) > 0:
    print("\nSample of duplicate products:")
    print("-" * 50)
    display(duplicates.head())
    
    # Remove duplicates keeping the first occurrence
    df.drop_duplicates(subset=['product_name', 'discounted_price'], keep='first', inplace=True)

# Verify no missing values remain
print("\nRemaining missing values after cleaning:")
print("-" * 50)
print(df.isnull().sum())

print("\nFinal dataset shape:", df.shape)


In [ ]:
### 6. Export Clean Data
- Save the cleaned dataset to a new CSV file
- Create a data quality report


In [ ]:
# Save cleaned dataset
df.to_csv('amazon_cleaned.csv', index=False)

# Generate data quality report
print("Data Quality Report")
print("=" * 50)

print("\n1. Dataset Dimensions:")
print(f"- Number of records: {len(df)}")
print(f"- Number of columns: {len(df.columns)}")

print("\n2. Column Data Types:")
print(df.dtypes)

print("\n3. Value Ranges:")
numeric_cols = ['discounted_price', 'actual_price', 'discount_percentage', 
                'rating', 'rating_count', 'savings_amount', 'savings_percentage']
for col in numeric_cols:
    print(f"\n{col}:")
    print(df[col].describe())

print("\n4. Category Distributions:")
categorical_cols = ['price_range', 'discount_range']
for col in categorical_cols:
    print(f"\n{col}:")
    print(df[col].value_counts(normalize=True).round(3) * 100, "%")

print("\n5. Data Completeness:")
print("All columns are now complete with no missing values.")

print("\n6. File Information:")
print(f"- Output file: amazon_cleaned.csv")
print(f"- File size: {round(df.memory_usage().sum() / (1024*1024), 2)} MB")
